# Initialising the Health Education England Agent

## If you wish to start at the beginning of the Doctor's in Training flow go [here](http://localhost:8889/lab/tree/Part%201%20-%20Getting%20Started.ipynb).

In this notebook you will register the DID of the HEE agent on the Sovrin StagingNet and write the credential definitions that Health Education England is responsible for issuing.

There are:
* Training Program: Schema_id - [2yWoMSfeTtYBFpUJFcnpqL:2:Training Programme:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141258)
* Training Program Placement: Schema id - [2yWoMSfeTtYBFpUJFcnpqL:2:Training Programme Placement:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141276)



## 1. Initialise a controller for HEE Agent

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8042
ADMIN_URL = "http://hee-agent:8041"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

## 2. Get current public DID

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

In [ ]:
response = await agent_controller.wallet.get_public_did()
print(response)

## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [ ]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

## 4. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [ ]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

## 5. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [ ]:
response = await agent_controller.ledger.get_taa()
TAA_text = response['result']['taa_record']['text']
TAA_version = response['result']['taa_record']['version']
TAA_digest = response['result']['taa_record']['digest']


taa_record = {
            "version": TAA_version,
            "mechanism": "service_agreement",
            'text': TAA_text
}


In [ ]:
response = await agent_controller.ledger.accept_taa(taa_record)
## Will return {} if successful
print(response)

## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [ ]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

## 7. Write Credential Definition for the Training Program to Ledger

The Health Education England must have a credential definition on the ledger to issue this credential.



In [ ]:
training_program_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Training Programme:0.0.1"
response = await agent_controller.definitions.write_cred_def(training_program_schema_id)

training_program_cred_def_id = response["credential_definition_id"]
print(training_program_cred_def_id)

%store training_program_schema_id
%store training_program_cred_def_id

## 8. Write Credential Definition for the Right to Work schema  to Ledger

The Health Education England must have a credential definition on the ledger to issue this credential.

In [ ]:
right_to_work_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Right to Work:0.0.1"
response = await agent_controller.definitions.write_cred_def(right_to_work_schema_id)

right_to_work_cred_def_id = response["credential_definition_id"]
print(right_to_work_cred_def_id)

%store right_to_work_schema_id
%store right_to_work_cred_def_id

## 9. Write Credential Definition for the Training Program Placement schema

The Health Education England must have a credential definition on the ledger to issue this credential.


In [ ]:
training_placement_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Training Programme Placement:0.0.1"
response = await agent_controller.definitions.write_cred_def(training_placement_schema_id)
training_placement_cred_def_id = response["credential_definition_id"]
print(training_placement_cred_def_id)

%store training_placement_schema_id
%store training_placement_cred_def_id

## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

# Proceed to [Part 4.2](http://localhost:8891/notebooks/Part%204.2%20-%20Interact%20with%20Doctor%20in%20Training.ipynb) where you will interact with a Doctor in Training.

Typically when developing an SSI application, this configuration only needs to happen once. Then the agent would be ready to issue these credentials to all future doctor interactions.
